In [1]:
#M01-T02: Exploració i estructura de les dades - Dataframes amb Pandas

In [2]:
#Importamos módulo
import pandas as pd
import numpy as np

In [3]:
#Exercisi 1- Descarrega el data set Airlines Delay: Airline on-time statistics and delay causes i carrega’l a un pandas Dataframe. 
#Explora les dades que conté, i queda’t únicament amb les columnes que consideris rellevants.

#Importamos archivo csv a una dataframe
df= pd.read_csv('DelayedFlights.csv', index_col=0) #Designamos la primera columna como un indice

In [4]:
print(df.head()) #print de unas filas para tener una referencia visual
print(df.info()) #print de informacion básica
print(df.shape) #1936758 filas y 29 columnas

   Year  Month  DayofMonth  DayOfWeek  DepTime  CRSDepTime  ArrTime  \
0  2008      1           3          4   2003.0        1955   2211.0   
1  2008      1           3          4    754.0         735   1002.0   
2  2008      1           3          4    628.0         620    804.0   
4  2008      1           3          4   1829.0        1755   1959.0   
5  2008      1           3          4   1940.0        1915   2121.0   

   CRSArrTime UniqueCarrier  FlightNum  ... TaxiIn  TaxiOut  Cancelled  \
0        2225            WN        335  ...    4.0      8.0          0   
1        1000            WN       3231  ...    5.0     10.0          0   
2         750            WN        448  ...    3.0     17.0          0   
4        1925            WN       3920  ...    3.0     10.0          0   
5        2110            WN        378  ...    4.0     10.0          0   

   CancellationCode  Diverted  CarrierDelay WeatherDelay NASDelay  \
0                 N         0           NaN          NaN   

In [5]:
#Check filas duplicadas:
temp_df= df.append(df) #duplicamos df agregándolo a él mismo
temp_df=temp_df.drop_duplicates().reset_index() #eliminamos duplicados
print(temp_df.shape) #Miramos cuantas filas hay-->1.936.756, 2 filas menos que la df original.


(1936756, 30)


In [6]:
print(df['Year'].unique()) #Todos los datos son del 2008 asique no nos hace falta esta columna

#Nueva df con las columnas relevantes
df_new= temp_df[['Month','UniqueCarrier','AirTime','ArrDelay','DepDelay','Origin','Dest','Distance','Cancelled','Diverted']]
print(df_new.head())

[2008]
   Month UniqueCarrier  AirTime  ArrDelay  DepDelay Origin Dest  Distance  \
0      1            WN    116.0     -14.0       8.0    IAD  TPA       810   
1      1            WN    113.0       2.0      19.0    IAD  TPA       810   
2      1            WN     76.0      14.0       8.0    IND  BWI       515   
3      1            WN     77.0      34.0      34.0    IND  BWI       515   
4      1            WN     87.0      11.0      25.0    IND  JAX       688   

   Cancelled  Diverted  
0          0         0  
1          0         0  
2          0         0  
3          0         0  
4          0         0  


In [7]:
#Exercisi2-Fes un informe complet del data set:.

#A)Resumeix estadísticament les columnes d’interès
print(df_new.describe()) #resumen estadístico de las variables continuas

#Variables categóricas= UniqueCarrier, Origin, Dest.
df_new['UniqueCarrier'].describe() #20 carriers diferentes (top:WN)

              Month       AirTime      ArrDelay      DepDelay      Distance  \
count  1.936756e+06  1.928369e+06  1.928369e+06  1.936756e+06  1.936756e+06   
mean   6.111111e+00  1.082772e+02  4.219988e+01  4.318517e+01  7.656863e+02   
std    3.482546e+00  6.864264e+01  5.678474e+01  5.340253e+01  5.744799e+02   
min    1.000000e+00  0.000000e+00 -1.090000e+02  6.000000e+00  1.100000e+01   
25%    3.000000e+00  5.800000e+01  9.000000e+00  1.200000e+01  3.380000e+02   
50%    6.000000e+00  9.000000e+01  2.400000e+01  2.400000e+01  6.060000e+02   
75%    9.000000e+00  1.370000e+02  5.600000e+01  5.300000e+01  9.980000e+02   
max    1.200000e+01  1.091000e+03  2.461000e+03  2.467000e+03  4.962000e+03   

          Cancelled      Diverted  
count  1.936756e+06  1.936756e+06  
mean   3.268352e-04  4.003602e-03  
std    1.807563e-02  6.314725e-02  
min    0.000000e+00  0.000000e+00  
25%    0.000000e+00  0.000000e+00  
50%    0.000000e+00  0.000000e+00  
75%    0.000000e+00  0.000000e+00  


count     1936756
unique         20
top            WN
freq       377602
Name: UniqueCarrier, dtype: object

In [12]:
#Troba quantes dades faltants hi ha per columna

print(df_new.isnull().sum())

#8387 filas sin Airtime ni ArrDelay == nº vuelos Cancelados + nºvuelos Desviados--> eliminaremos estas filas porque no son relevantes para calcular vuelos retrasados
print(df_new['Cancelled'].sum()+df_new['Diverted'].sum())

Canc_dev_idx=df_new[(df_new['Cancelled']==1) | (df_new['Diverted']==1)].index #Buscamos el índice de las filas con vuelos cancelados o desviados
df_new2=df_new.drop(Canc_dev_idx).reset_index() #Eliminamos estas filas

print(df_new2.isnull().sum()) #La nueva df sin vuelos cancelados/desviados no tiene null en Airtime/Arr.delay

Month               0
UniqueCarrier       0
AirTime          8387
ArrDelay         8387
DepDelay            0
Origin              0
Dest                0
Distance            0
Cancelled           0
Diverted            0
dtype: int64
8387
index            0
Month            0
UniqueCarrier    0
AirTime          0
ArrDelay         0
DepDelay         0
Origin           0
Dest             0
Distance         0
Cancelled        0
Diverted         0
dtype: int64


In [9]:
#Crea columnes noves (speed, delayed, ruta origen-destino)

#Columna nueva con la velocidad media por vuelo.
df_new2['Speed']=df_new2['Distance']/df_new2['AirTime'] 

#Columna nueva para saber si el vuelo llegó con retraso o no (1=Delayed)
def delayed(ArrDelay): #Creamos una funcion que asigna 1 a la columna "Delayed" si se retraso al llegar (se considera que los valores negativos significan que llegó antes de lo previsto)
    if ArrDelay>15: #Dejaremos 15 de margen para identificar si esta retrasado o no
        return 1
    else:
        return 0

df_new2['Delayed']=df_new2['ArrDelay'].apply(delayed)
        
#Columna nueva que nos indique la ruta del vuelo.
df_new2['Route']=df_new2['Origin']+"-"+df_new2['Dest'] 
        
print(df_new2)

           index  Month UniqueCarrier  AirTime  ArrDelay  DepDelay Origin  \
0              0      1            WN    116.0     -14.0       8.0    IAD   
1              1      1            WN    113.0       2.0      19.0    IAD   
2              2      1            WN     76.0      14.0       8.0    IND   
3              3      1            WN     77.0      34.0      34.0    IND   
4              4      1            WN     87.0      11.0      25.0    IND   
...          ...    ...           ...      ...       ...       ...    ...   
1928364  1936751     12            DL    120.0      25.0      30.0    MSP   
1928365  1936752     12            DL     78.0      75.0      57.0    RIC   
1928366  1936753     12            DL    122.0      99.0      80.0    ATL   
1928367  1936754     12            DL     89.0       9.0      11.0    IAD   
1928368  1936755     12            DL    104.0      -5.0       7.0    SAT   

        Dest  Distance  Cancelled  Diverted     Speed  Delayed    Route  
0

In [10]:
#Taula de les aerolínies amb més endarreriments acumulats

#Creamos una pivot table que nos indique el nºvuelos retrasados y nºvuelos totales.
#Para calcular nºvuelos totales contamos cuantos vuelos tenemos.

pivot_df= df_new2.pivot_table(index='UniqueCarrier',
                              values=["Delayed","Month"],
                              aggfunc={'Delayed':np.sum,
                                       'Month':'count'}).sort_values(by = ['Delayed'],ascending = False).reset_index()

pivot_df=pivot_df.rename(columns={'Month':'Total'})

print(pivot_df) #WN es la aerolínea con más vuelos retrasados pero tambien es la que más vuelos realiza.

#Creamos una nueva columna que nos diga el % de vuelos retrasados por aerolínea

pivot_df['Delayed(%)']=((pivot_df['Delayed']/pivot_df['Total'])*100 )

Delay_carrier=pivot_df.sort_values(by=['Delayed(%)'],ascending=False).reset_index() 

print(Delay_carrier) #El 74% de los vuelos de YV llegan con retraso.


   UniqueCarrier  Delayed   Total
0             WN   196424  376201
1             AA   129401  190910
2             MQ    95126  141223
3             UA    93355  140904
4             OO    86619  131780
5             XE    70455  103147
6             DL    70260  113728
7             US    57936   98007
8             CO    57497   99731
9             EV    55483   81762
10            NW    53392   78843
11            YV    49536   66769
12            FL    45738   70969
13            OH    38362   52453
14            B6    37538   54925
15            9E    34836   51569
16            AS    23340   39010
17            F9    15267   28222
18            HA     4124    7472
19            AQ      321     744
    index UniqueCarrier  Delayed   Total  Delayed(%)
0      11            YV    49536   66769   74.190118
1      13            OH    38362   52453   73.135950
2      14            B6    37538   54925   68.344106
3       5            XE    70455  103147   68.305428
4       9            

In [11]:
#Quins són els vols més llargs? I els més endarrerits? 

#¿Qúe ruta tiene más retrasos?
Route_Delay=df_new2.groupby(['Route'])['Delayed'].sum().sort_values(ascending=False) #Agrupamos por ruta la suma de vuelos retrasados.
print(Route_Delay)
#Vuelos con más retrasos: LAX-SFO, ORD-LGA,SFO-LAX

#¿Cuales son los vuelos más largos?
Route_AirTime=df_new2.groupby(['Route'])['AirTime'].mean().sort_values(ascending=False) #Agrupamos por ruta la media de tiempo de vuelo de cada ruta
print(Route_AirTime)
#Vuelos más largos de media: EWR-HNL, ATL-HNL, HNL-EWR


Route
LAX-SFO    3221
ORD-LGA    3206
SFO-LAX    2779
ATL-LGA    2722
LGA-ORD    2598
           ... 
BZN-DTW       0
SCE-CVG       0
MKE-SLC       0
ATL-TUP       0
LEX-CLE       0
Name: Delayed, Length: 5127, dtype: int64
Route
EWR-HNL    607.475472
ATL-HNL    552.555556
HNL-EWR    544.087719
ORD-HNL    517.135922
ORD-OGG    502.040541
              ...    
WRG-PSG     11.918367
SJC-SFO     11.000000
IAD-DCA     11.000000
HPN-EWR     11.000000
IAH-HOU     10.000000
Name: AirTime, Length: 5127, dtype: float64
